In [68]:
import pandas as pd

In [69]:
df = pd.read_csv('newark_flights.csv')

In [70]:
df.columns

Index(['Unnamed: 0', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE',
       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [71]:
df.head()

,Unnamed: 0,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,37,2015,1,1,4,UA,1528,N76519,SJU,EWR,...,458.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,45,2015,1,1,4,B6,1990,N597JB,SJU,EWR,...,516.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,49,2015,1,1,4,UA,1162,N37293,BQN,EWR,...,605.0,6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,110,2015,1,1,4,EV,4160,N11150,JAX,EWR,...,743.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,125,2015,1,1,4,EV,4646,N29917,CHS,EWR,...,742.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [73]:
df.isna().sum()

YEAR                        0
MONTH                       0
DAY                         0
DAY_OF_WEEK                 0
AIRLINE                     0
FLIGHT_NUMBER               0
TAIL_NUMBER              1802
ORIGIN_AIRPORT              0
DESTINATION_AIRPORT         0
SCHEDULED_DEPARTURE         0
DEPARTURE_TIME           6282
DEPARTURE_DELAY          6282
TAXI_OUT                 6447
WHEELS_OFF               6447
SCHEDULED_TIME              0
ELAPSED_TIME             7038
AIR_TIME                 7038
DISTANCE                    0
WHEELS_ON                6531
TAXI_IN                  6531
SCHEDULED_ARRIVAL           0
ARRIVAL_TIME             6531
ARRIVAL_DELAY            7038
DIVERTED                    0
CANCELLED                   0
CANCELLATION_REASON    197142
AIR_SYSTEM_DELAY       160388
SECURITY_DELAY         160388
AIRLINE_DELAY          160388
LATE_AIRCRAFT_DELAY    160388
WEATHER_DELAY          160388
dtype: int64

In [74]:
df.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

In [75]:
for col in df.columns:
    print(col, '\n', df[col].value_counts(normalize=True).head(), '\n\n')

YEAR 
 2015    1.0
Name: YEAR, dtype: float64 


MONTH 
 7     0.097145
12    0.095274
8     0.094582
6     0.094400
3     0.093963
Name: MONTH, dtype: float64 


DAY 
 16    0.034366
9     0.034145
2     0.033973
17    0.033693
10    0.033472
Name: DAY, dtype: float64 


DAY_OF_WEEK 
 1    0.152665
3    0.152366
4    0.150657
2    0.150647
5    0.147749
Name: DAY_OF_WEEK, dtype: float64 


AIRLINE 
 UA    0.422216
EV    0.310557
B6    0.062391
WN    0.053899
AA    0.053054
Name: AIRLINE, dtype: float64 


FLIGHT_NUMBER 
 673     0.003369
14      0.003247
1942    0.003080
174     0.002942
1162    0.002937
Name: FLIGHT_NUMBER, dtype: float64 


TAIL_NUMBER 
 N14907    0.002641
N12900    0.002299
N18556    0.002225
N26549    0.002161
N14916    0.002156
Name: TAIL_NUMBER, dtype: float64 


ORIGIN_AIRPORT 
 EWR    0.499858
ATL    0.025005
SFO    0.024681
MCO    0.023526
LAX    0.022682
Name: ORIGIN_AIRPORT, dtype: float64 


DESTINATION_AIRPORT 
 EWR    0.500142
ATL    0.025034
SFO    0.02